In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

# Grad-CAM++ Function
def gradCAM_plus_plus(img, model, last_conv_layer_name):
    grad_model = tf.keras.models.Model(
        inputs=[model.inputs],
        outputs=[model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape1:
        with tf.GradientTape() as tape2:
            with tf.GradientTape() as tape3:
                conv_outputs, predictions = grad_model(img)
                pred_index = tf.argmax(predictions[0])
                class_channel = predictions[:, pred_index]

            grads = tape3.gradient(class_channel, conv_outputs)
        first_derivative = grads
        second_derivative = tape2.gradient(grads, conv_outputs)
    third_derivative = tape1.gradient(second_derivative, conv_outputs)

    conv_outputs = conv_outputs[0]
    first_derivative = first_derivative[0]
    second_derivative = second_derivative[0]
    third_derivative = third_derivative[0]

    global_sum = np.sum(conv_outputs.numpy(), axis=(0, 1))
    alpha_num = second_derivative.numpy()
    alpha_denom = 2 * second_derivative.numpy() + third_derivative.numpy() * global_sum[np.newaxis, np.newaxis, :]
    alpha_denom = np.where(alpha_denom != 0.0, alpha_denom, 1e-10)

    alphas = alpha_num / alpha_denom
    weights = np.maximum(first_derivative.numpy(), 0.0)
    deep_linearization_weights = np.sum(alphas * weights, axis=(0, 1))

    heatmap = np.sum(deep_linearization_weights * conv_outputs.numpy(), axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap) + 1e-8

    return heatmap

# Outline with Adaptive Threshold
def draw_gradcam_fill_and_outline(heatmap, original_img, threshold_ratio=0.5, 
                                   fill_color=(255, 150, 150), outline_color=(0, 255, 0),
                                   fill_alpha=0.4, outline_thickness=1):
    if original_img.max() <= 1.0:
        img_uint8 = (original_img * 255).astype(np.uint8)
    else:
        img_uint8 = original_img.copy()

    heatmap_resized = cv2.resize(heatmap, (img_uint8.shape[1], img_uint8.shape[0]))
    heatmap_resized = np.uint8(255 * heatmap_resized)
    threshold = int(threshold_ratio * 255)
    _, binary_mask = cv2.threshold(heatmap_resized, threshold, 255, cv2.THRESH_BINARY)

    kernel = np.ones((5, 5), np.uint8)
    cleaned_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_OPEN, kernel)
    cleaned_mask = cv2.morphologyEx(cleaned_mask, cv2.MORPH_CLOSE, kernel)

    overlay = np.full_like(img_uint8, fill_color, dtype=np.uint8)
    mask_3ch = np.stack([cleaned_mask]*3, axis=-1) // 255
    filled_img = np.where(mask_3ch, cv2.addWeighted(img_uint8, 1 - fill_alpha, overlay, fill_alpha, 0), img_uint8)

    contours, _ = cv2.findContours(cleaned_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    outlined = cv2.drawContours(filled_img.copy(), contours, -1, outline_color, outline_thickness)

    return (outlined / 255.0) if original_img.max() <= 1.0 else outlined


In [ ]:
""" # Extract true class name from file path (modify based on your folder structure)
        true_class = image_names[i]
        
        # Load and preprocess
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0) / 255.0

        # Predict class
        preds = model_e_cbam.predict(img_array)
        pred_idx = np.argmax(preds)
        pred_class = class_names[pred_idx]

        print(f"[{i+1}] True Class: {true_class} | Predicted Class: {pred_class}")

        # Grad-CAM++
        heatmap = gradCAM_plus_plus(img_array, model_e_cbam, last_conv_layer_name)
        
        percentile = 85
        threshold_value = np.percentile(heatmap, percentile)
        threshold_ratio = threshold_value / (np.max(heatmap) + 1e-8)

        outlined_img = draw_gradcam_fill_and_outline(heatmap, img_array[0], threshold_ratio=threshold_ratio)

        plt.figure(figsize=(12, 5))
        plt.suptitle(f"True: {true_class} | Predicted: {pred_class}", fontsize=14)

        plt.subplot(1, 3, 1)
        plt.title("Original Image")
        plt.imshow(img_array[0])
        plt.axis('off')

        plt.subplot(1, 3, 2)
        plt.title("Grad-CAM++ Heatmap")
        plt.imshow(heatmap, cmap='jet')
        plt.axis('off')

        plt.subplot(1, 3, 3)
        plt.title("Outlined Grad-CAM++ Region")
        plt.imshow(outlined_img)
        plt.axis('off')

        plt.tight_layout()
        plt.savefig(f'gradcam_output_{i+1}.jpg')
        plt.show()"""